# 2.4A
> 2.4A

- toc: true 
- badges: true
- comments: true
- categories: [Posts]
- title: 2.4A
- layouts:base

# Imports and Flask Objects
Defines and key object creations

Comment on where you have observed these working? Provide a defintion of purpose.
Flask app object: The Flask app object is an instance of the Flask class that represents a Flask web application. It is used to define routes, views, and other application components.
SQLAlchemy db object: SQLAlchemy db object, on the other hand, is an instance of the SQLAlchemy database class. SQLAlchemy is an Object-Relational Mapping (ORM) tool that provides a high-level interface for interacting with relational databases. 

In [ ]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

# Model Definition
Define columns, initialization, and CRUD methods for users table in sqlite.db

Comment on these items in the class, purpose and defintion.
class User- The class user is where you initiliaze and create the sql database of the users
db.Model inheritance-this allows you to get db properties from above in the code
init method- the init method is used to initiliaze the database and create it
@property, @<column>.setter- These are ways to define the property, they are very useful because they allow us to see the purpose of what each command is doing. 
create, read, update, delete methods- These are all ways to manipulate the database, namely with create it means you add a user, read is reading the database, update is updating the database with new information, delete is where you delete a user from the database. 

In [ ]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, a string represents date outside object
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob setter, verifies date type before it is set or default to today
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.add(self) # performs update when id exists
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

# Initial Data
Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

Comment on how these work?
Create All Tables from db Object-the Db is how we create the sql table, we use db.createall() to create the table with the users 1-6
User Object Constructors: These are used to set the information for each of the users specifically
Try / Except: Try/except is used to show what should happen and if it doesnt happen then check for a certain error, printing something that will show that error. 

In [ ]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

# Check for given Credentials in users table in sqlite.db
Use of ORM Query object and custom methods to identify user to credentials uid and password

Comment on purpose of following
User.query.filter_by - This filters through and prints the uid in a sequential order
user.password- this is used to check the password, which is used in authenticating the user as a real user .

In [ ]:
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

# Create a new User in table in Sqlite.db
Uses SQLALchemy and custom user.create() method to add row.

Comment on purpose of following
user.find_by_uid() and try/except - this is how we search through the table looking for specifically a user with a matching uid
user = User(...) - This is how we initilize the user object, by setting the name uid and passwords for the new user
user.dob and try/except-We use try and except here becuase it allows us to make sure a valid date is enetered when creating the user, its important to have a real date in the table.  
user.create() and try/except - This is used to check if the users uid has an issue when creating if nothing is printed thats good, if something is then the user has an unkown uid 

In [ ]:
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

# Reading users table in sqlite.db
Uses SQLALchemy query.all method to read data

Comment on purpose of following
User.query.all - This is how we pool our data from the table, the query.all() is used to check through all users
json_ready assignment, google List Comprehension - The purpose of list comprhension is creating a new list with existing values and this what we do when we read the table. 

In [ ]:
# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

# Hacks

In [2]:
import sqlite3

# create connection object and cursor
conn = sqlite3.connect('files/sports.db')
c = conn.cursor()

# create table for players
c.execute('''CREATE TABLE players
             (id INTEGER PRIMARY KEY,
             name TEXT,
             sport TEXT,
             team TEXT,
             age INTEGER)''')

# create table for teams
c.execute('''CREATE TABLE teams
             (id INTEGER PRIMARY KEY,
             name TEXT,
             sport TEXT,
             city TEXT)''')

# insert sample data for players
c.execute("INSERT INTO players VALUES (1, 'LeBron James', 'Basketball', 'Los Angeles Lakers', 37)")
c.execute("INSERT INTO players VALUES (2, 'Tom Brady', 'Football', 'Tampa Bay Buccaneers', 44)")
c.execute("INSERT INTO players VALUES (3, 'Serena Williams', 'Tennis', 'N/A', 40)")

# insert sample data for teams
c.execute("INSERT INTO teams VALUES (1, 'Los Angeles Lakers', 'Basketball', 'Los Angeles')")
c.execute("INSERT INTO teams VALUES (2, 'Tampa Bay Buccaneers', 'Football', 'Tampa Bay')")
c.execute("INSERT INTO teams VALUES (3, 'New York Yankees', 'Baseball', 'New York City')")

# commit changes and close connection
conn.commit()
conn.close()

# Create

In [4]:
import sqlite3

def create_player(name, sport, team, age):
    # connect to database
    conn = sqlite3.connect('files/sports.db')
    c = conn.cursor()

    # insert new player data
    c.execute("INSERT INTO players (name, sport, team, age) VALUES (?, ?, ?, ?)",
              (name, sport, team, age))

    # commit changes and close connection
    conn.commit()
    conn.close()

# example usage
create_player('Christiano Ronaldo', 'Soccer', 'Manchester United', 37)


# Read


In [5]:
import sqlite3

def read_players():
    # connect to database
    conn = sqlite3.connect('files/sports.db')
    c = conn.cursor()

    # select all players from the players table
    c.execute("SELECT * FROM players")
    players = c.fetchall()

    # print player data
    for player in players:
        print(player)

    # close connection
    conn.close()

# example usage
read_players()


(1, 'LeBron James', 'Basketball', 'Los Angeles Lakers', 37)
(2, 'Tom Brady', 'Football', 'Tampa Bay Buccaneers', 44)
(3, 'Serena Williams', 'Tennis', 'N/A', 40)
(4, 'Cristiano Ronaldo', 'Soccer', 'Manchester United', 37)


# Update

In [7]:
import sqlite3

def update_player_age(name, new_age):
    # connect to database
    conn = sqlite3.connect('files/sports.db')
    c = conn.cursor()

    # update player age
    c.execute("UPDATE players SET age = ? WHERE name = ?", (new_age, name))

    # commit changes and close connection
    conn.commit()
    conn.close()

# example usage
update_player_age('LeBron James', 80)


# Delete

In [8]:
import sqlite3

def delete_player(name):
    # connect to database
    conn = sqlite3.connect('files/sports.db')
    c = conn.cursor()

    # delete player
    c.execute("DELETE FROM players WHERE name = ?", (name,))

    # commit changes and close connection
    conn.commit()
    conn.close()

# example usage
delete_player('Serena Williams')
